In [ ]:
#import necessary packages
import  numpy as np
import tensorflow as tf
from model import build_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

2025-06-10 16:19:07.303246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#load the training and validation data
allImagesNp = np.load("data/Unet-Train-Lung-Images.npy")
maskImagesNp = np.load("data/Unet-Train-Lung-Masks.npy")

allValidateImagesNP = np.load("data/Unet-Validate-Lung-Images.npy")
maskValidateImagesNP = np.load("data/Unet-Validate-Lung-Masks.npy")

In [ ]:
##UNCOMMENT to train using wavelet generated masks (optimized using darwinSet)

# wavelet = np.load("data_wavelet_darwin/Darwin_0.059.npy")
# allImagesNp = np.load("data_wavelet_darwin/Unet-Train-Lung-Images-Darwin.npy")
# maskImagesNp = wavelet[:5495]

# allValidateImagesNP = np.load("data_wavelet_darwin/Unet-Validate-Lung-Images-Darwin.npy")
# maskValidateImagesNP = wavelet[5495:]

In [ ]:
##UNCOMMENT to train using wavelet generated masks (optimized using shenzhenSet)

# wavelet = np.load("data_wavelet_shenzhen/Shenzhen_0.082.npy")
# allImagesNp = np.load("data_wavelet_shenzhen/Unet-Train-Lung-Images-Shenzhen.npy")
# maskImagesNp = wavelet[:509]

# allValidateImagesNP = np.load("data_wavelet_shenzhen/Unet-Validate-Lung-Images-Shenzhen.npy")
# maskValidateImagesNP = wavelet[509:]

In [ ]:
#recheck the shape of the loaded data
print(allImagesNp.shape)
print(maskImagesNp.shape)
print(allValidateImagesNP.shape)
print(maskValidateImagesNP.shape)

(6129, 256, 256, 3)
(6129, 256, 256)
(681, 256, 256, 3)
(681, 256, 256)


In [ ]:
#set the image resolution
height = 256
width = 256

In [ ]:
#initialize the parameters
shape = (height, width, 3)
lr = 1e-4
batchSize = 4
epochs = 50

In [ ]:
model = build_model(shape)

In [8]:
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 31,402,497 (119.79 MB)

 Trainable params: 31,390,721 (119.75 MB)

 Non-trainable params: 11,776 (46.00 KB)

None


In [9]:
opt = tf.keras.optimizers.Adam(lr)

In [10]:
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [11]:
stepsPerEpoch = int(np.ceil(len(allImagesNp)/batchSize))
validationSteps = int(np.ceil(len(allValidateImagesNP)/batchSize))

In [ ]:
#save the best model during training
best_model_file = "temp/best-lung-Unet-aggregated.keras"

#rename accordingly based on the training dataset used
#best_model_file = "temp/best-lung-Unet-darwin.keras"
#best_model_file = "temp/best-lung-Unet-shenzhen.keras"
#best_model_file = "temp/best-lung-Unet-waveletdarwin.keras"
#best_model_file = "temp/best-lung-Unet-waveletshenzhen.keras"

In [13]:
callbacks = [ModelCheckpoint(best_model_file, verbose=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.1, verbose=1, min_lr=1e-7),
             EarlyStopping(monitor='val_accuracy', patience=20, verbose=1)]

In [14]:
history = model.fit(allImagesNp, maskImagesNp,
                    batch_size = batchSize,
                    epochs = epochs,
                    verbose =1,
                    validation_data = (allValidateImagesNP, maskValidateImagesNP),
                    validation_steps = validationSteps,
                    steps_per_epoch = stepsPerEpoch,
                    shuffle = True,
                    callbacks = callbacks)

Epoch 1/50
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9423 - loss: 0.1474
Epoch 1: val_loss improved from inf to 0.08855, saving model to temp/best-lung-Unet-aggregated.keras
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 5159s 3s/step - accuracy: 0.9423 - loss: 0.1473 - val_accuracy: 0.9653 - val_loss: 0.0886 - learning_rate: 1.0000e-04
Epoch 2/50
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9680 - loss: 0.0812
Epoch 2: val_loss did not improve from 0.08855
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 5216s 3s/step - accuracy: 0.9680 - loss: 0.0812 - val_accuracy: 0.8989 - val_loss: 0.2924 - learning_rate: 1.0000e-04
Epoch 3/50
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9691 - loss: 0.0774
Epoch 3: val_loss improved from 0.08855 to 0.07669, saving model to temp/best-lung-Unet-aggregated.keras
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 5261s 3s/step - accuracy: 0.9691 - loss: 0.0774 - val_accuracy: 0.9690 - val_loss: 0.0767 - learning_rate: 1.0000e-04
Epoch 4/50
1533/1533 ━━━━━━━━━━━━━━━━━━